In [3]:
from pathlib import Path
import json
from datetime import datetime
from typing import List, Dict, Any, Optional

from graphrag_sdk import Attribute, AttributeType, Entity, Ontology, Relation
from autogen.agentchat.contrib.graph_rag.document import Document, DocumentType
from autogen.agentchat.contrib.graph_rag.falkor_graph_query_engine import FalkorGraphQueryEngine
from autogen.agentchat.contrib.graph_rag.falkor_graph_rag_capability import FalkorGraphRagCapability
from autogen import ConversableAgent

class MemoryGraphEngine:
    """Knowledge graph engine for maintaining conversation memory"""
    
    def __init__(self, name: str = "memory_graph", host: str = "localhost", port: int = 6379):
        # Create knowledge base directory
        self.memory_dir = Path("memory_store")
        self.memory_dir.mkdir(exist_ok=True)
        
        # Create query engine
        self.query_engine = FalkorGraphQueryEngine(
            name=name,
            host=host,
            port=port
        )
        
        # Initialize with empty memory if needed
        self._init_if_empty()
        
    def _init_if_empty(self):
        """Initialize memory store if empty"""
        if not list(self.memory_dir.glob("*.txt")):
            empty_doc = self._create_memory_file({
                "type": "memory_init",
                "timestamp": datetime.now().isoformat()
            })
            self.query_engine.init_db(input_doc=[empty_doc])
    
    def _create_memory_file(self, content: Dict) -> Document:
        """Create a memory document file"""
        timestamp = datetime.now().strftime("%Y%m%d_%H%M%S")
        filepath = self.memory_dir / f"memory_{timestamp}.txt"
        
        # Save formatted content
        with open(filepath, 'w') as f:
            json.dump(content, f, indent=2)
            
        return Document(doctype=DocumentType.TEXT, path_or_url=str(filepath))

    def store_memory(self, summary: str, topics: List[str], facts: List[Dict[str, Any]], 
                    prev_conversation_id: Optional[str] = None) -> str:
        """Store new memory as a document"""
        timestamp = datetime.now().isoformat()
        
        # Format memory content
        memory_content = {
            "type": "memory",
            "timestamp": timestamp,
            "summary": summary,
            "topics": topics,
            "facts": facts,
            "previous_memory": prev_conversation_id
        }
        
        # Create memory document
        memory_doc = self._create_memory_file(memory_content)
        
        # Initialize DB if this is first memory, otherwise load as new document
        if len(list(self.memory_dir.glob("*.txt"))) == 1:
            self.query_engine.init_db(input_doc=[memory_doc])
        else:
            existing_docs = [
                Document(doctype=DocumentType.TEXT, path_or_url=str(p))
                for p in self.memory_dir.glob("*.txt")
            ]
            self.query_engine.init_db(input_doc=existing_docs)
        
        return timestamp

    async def query_knowledge(self, query: str) -> List[Dict[str, Any]]:
        """Query the knowledge base"""
        return await self.query_engine.query(query)

def create_memory_agent(
    name: str = "memory_agent",
    llm_config: Optional[Dict[str, Any]] = None,
    memory_config: Optional[Dict[str, Any]] = None
) -> ConversableAgent:
    """Create an agent with memory capability"""
    
    # Create base agent
    agent = ConversableAgent(
        name=name,
        llm_config=llm_config,
        human_input_mode="NEVER",
        system_message="""You are a helpful agent with long-term memory capabilities.
        You can store and recall information using your memory.
        When asked about topics, try to reference relevant information from your memory."""
    )
    
    # Create memory engine
    memory_engine = MemoryGraphEngine(**(memory_config or {}))
    
    # Add graph capability
    graph_capability = FalkorGraphRagCapability(memory_engine.query_engine)
    graph_capability.add_to_agent(agent)
    
    # Add memory engine reference
    setattr(agent, "memory_engine", memory_engine)
    
    return agent

In [4]:
import os
import autogen
from autogen import UserProxyAgent

# Configure OpenAI
config_list = autogen.config_list_from_json(env_or_file="OAI_CONFIG_LIST")
os.environ["OPENAI_API_KEY"] = config_list[0]["api_key"]

# Create memory agent
memory_agent = create_memory_agent(
    name="memory_assistant",
    memory_config={
        "name": "test_memory",
        "host": "localhost",
        "port": 6380  # Match your FalkorDB port
    }
)

# Create user proxy
user_proxy = UserProxyAgent(
    name="user",
    human_input_mode="ALWAYS",
    code_execution_config=False
)

# Store quantum computing knowledge
quantum_knowledge = {
    "summary": "Introduction to quantum computing concepts",
    "topics": ["quantum computing", "qubits", "quantum gates"],
    "facts": [
        {
            "content": "Qubits can exist in multiple states simultaneously due to superposition",
            "confidence": 1.0,
            "topics": ["qubits", "quantum computing"]
        },
        {
            "content": "Quantum gates manipulate qubits to perform computations",
            "confidence": 1.0,
            "topics": ["quantum gates", "quantum computing"]
        }
    ]
}

# Store the knowledge
memory_agent.memory_engine.store_memory(**quantum_knowledge)

# Test retrieving knowledge
user_proxy.initiate_chat(
    memory_agent,
    message="What do you know about quantum computing?"
)

user (to memory_assistant):

What do you know about quantum computing?

--------------------------------------------------------------------------------
memory_assistant (to user):

I'm sorry, but I don't have information on quantum computing based on the provided context.

--------------------------------------------------------------------------------
user (to memory_assistant):

what is the provided context

--------------------------------------------------------------------------------
memory_assistant (to user):

The provided context includes information about memories related to the topic of quantum computing. Specifically, it mentions a memory titled "Introduction to quantum computing concepts," dated November 30, 2024. This memory covers concepts such as "Quantum Computing," "Qubits," and "Quantum Gates."

--------------------------------------------------------------------------------


ChatResult(chat_id=None, chat_history=[{'content': 'What do you know about quantum computing?', 'role': 'assistant', 'name': 'user'}, {'content': "I'm sorry, but I don't have information on quantum computing based on the provided context.", 'role': 'user', 'name': 'memory_assistant'}, {'content': 'what is the provided context', 'role': 'assistant', 'name': 'user'}, {'content': 'The provided context includes information about memories related to the topic of quantum computing. Specifically, it mentions a memory titled "Introduction to quantum computing concepts," dated November 30, 2024. This memory covers concepts such as "Quantum Computing," "Qubits," and "Quantum Gates."', 'role': 'user', 'name': 'memory_assistant'}], summary='The provided context includes information about memories related to the topic of quantum computing. Specifically, it mentions a memory titled "Introduction to quantum computing concepts," dated November 30, 2024. This memory covers concepts such as "Quantum Com